In [1]:
import numpy as np

## Formula

** Forward propagation **
$$O^{[1]} = W1^{[1]}State + B^{[1]}$$
$$G^{[1]} = {f_\text{ReLU}}(O^{[1]})$$
$$O^{[2]} = W^{[2]}G^{[1]} + B^{[2]}$$
$$G^{[2]} = {f_\text{ReLU}}(O^{[2]})$$
$$O^{[3]} = W^{[3]} G^{[2]} + B^{[3]}$$
$$Loss = (Y-O^{[3]})^{2}$$

**Shape**
$$\frac{d_\text{L}}{{d_\text{O3}}}: A * 1$$
$$\frac{d_\text{L}}{{d_\text{W3}}}: A * 128$$
$$\frac{d_\text{L}}{{d_\text{B3}}}: A * 1$$
$$\frac{d_\text{L}}{{d_\text{G2}}}: 128 * 1$$
$$\frac{d_\text{L}}{{d_\text{W2}}}: 128 * 128$$

**Backpropagation**
$$\frac{d_\text{L}}{{d_\text{O3}}} = 2(O^{[3]}-Y)$$
$$\frac{d_\text{L}}{{d_\text{W3}}} = \frac{d_\text{L}}{{d_\text{O3}}} \frac{d_\text{O3}}{{d_\text{W3}}} = \frac{d_\text{L}}{{d_\text{O3}}} G^{[2]T} {(*)}$$
$$\frac{d_\text{L}}{{d_\text{B3}}} = \frac{d_\text{L}}{{d_\text{O3}}} \frac{d_\text{O3}}{{d_\text{B3}}} = \frac{d_\text{L}}{{d_\text{O3}}} 1^{T} {(*)}$$
$$\frac{d_\text{L}}{{d_\text{G2}}} = \frac{d_\text{L}}{{d_\text{O3}}} \frac{d_\text{O3}}{{d_\text{G2}}} = W^{[3]T} \frac{d_\text{L}}{{d_\text{O3}}}$$
$$\frac{d_\text{L}}{{d_\text{W2}}} = \frac{d_\text{L}}{{d_\text{G2}}} \frac{d_\text{G2}}{{d_\text{O2}}}\frac{d_\text{O2}}{{d_\text{W2}}} = \frac{d_\text{L}}{{d_\text{G2}}} * {(O^{[2]}>0)} * {G^{[1]T}} {(*)}$$
$$\frac{d_\text{L}}{{d_\text{B2}}} = \frac{d_\text{L}}{{d_\text{G2}}} \frac{d_\text{G2}}{{d_\text{O2}}}\frac{d_\text{O2}}{{d_\text{B2}}} = \frac{d_\text{L}}{{d_\text{G2}}} * {(O^{[2]}>0)} * {1^{[1]T}} {(*)}$$
$$\frac{d_\text{L}}{{d_\text{G1}}} = \frac{d_\text{L}}{{d_\text{G2}}} \frac{d_\text{G2}}{{d_\text{O2}}}\frac{d_\text{O2}}{{d_\text{G1}}} ={W^{[2]T}}*\frac{d_\text{L}}{{d_\text{G2}}}*{(O^{[2]}>0)} $$
$$\frac{d_\text{L}}{{d_\text{W1}}} = \frac{d_\text{L}}{{d_\text{G1}}} \frac{d_\text{G1}}{{d_\text{O1}}} \frac{d_\text{O1}}{{d_\text{W1}}} = \frac{d_\text{L}}{{d_\text{G1}}} * {(O^{[1]}>0)} * State^{T} {(*)} $$
$$\frac{d_\text{L}}{{d_\text{B1}}} = \frac{d_\text{L}}{{d_\text{G1}}} \frac{d_\text{G1}}{{d_\text{O1}}} \frac{d_\text{O1}}{{d_\text{B1}}} = \frac{d_\text{L}}{{d_\text{G1}}} * {(O^{[1]}>0)} * 1^{T} {(*)} $$


**Update**
$${W^{[3]} = W^{[3]} - \frac{d_\text{L}}{{d_\text{W3}}}}$$
$${B^{[3]} = W^{[3]} - \frac{d_\text{L}}{{d_\text{B3}}}}$$
$${W^{[2]} = W^{[2]} - \frac{d_\text{L}}{{d_\text{W2}}}}$$
$${B^{[2]} = W^{[2]} - \frac{d_\text{L}}{{d_\text{B2}}}}$$
$${W^{[1]} = W^{[1]} - \frac{d_\text{L}}{{d_\text{W1}}}}$$
$${B^{[1]} = W^{[1]} - \frac{d_\text{L}}{{d_\text{B1}}}}$$

## Code

In [2]:
# @njit
def forward(W1,b1,W2,b2,W3,b3,X):
    O1 = W1 @ X.T + b1
    G1 = np.maximum(0,O1)
    O2 = W2 @ G1 + b2
    G2 = np.maximum(0,O2)
    O3 = W3 @ G2 + b3
    return O1,O2,O3

In [3]:
# @njit
def mean_numba(X):
    R = []
    for i in np.arange(X.shape[0]):
        R.append(X[i,:].mean())
    return np.array(R).reshape(-1,1) * 1.0

In [4]:
# @njit
def backward(W1,b1,W2,b2,W3,b3,S,Y,O1,O2,O3,lr):
    # print(O3.shape,Y.shape)
    dO3 = (O3-Y.T)
    dG2 =  W3.T.dot(dO3)
    dG1 = W2.T.dot(dG2 * (O2 >= 0))
    dW1 =  (dG1 * (O1 >= 0)).dot(S)
    dB1 = mean_numba(dG1 * (O1 >= 0))#np.mean(dG1 * (O1 > 0)) #
    dW2 = (dG2 * (O2 >= 0)).dot(np.maximum(0,O1).T)
    dB2 = mean_numba(dG2 * (O2 >= 0)) #dG2 * (O2 > 0) * np.ones_like(b2) / b2[0]#np.mean(dG2 * (O2 > 0))#
    dW3 = dO3.dot(np.maximum(0,O2).T)
    dB3 = mean_numba(dO3) #dO3 * np.ones_like(b3) / b3[0] #np.mean(dO3)#
    # print(dB1,dW1,dW2,dB2,dW3,dB3)
    # print()
    W1 = W1 - dW1*lr
    b1 = b1 - dB1*lr
    W2 = W2 - dW2*lr
    b2 = b2 - dB2*lr
    W3 = W3 - dW3*lr
    b3 = b3 - dB3*lr
    return W1,b1,W2,b2,W3,b3

In [5]:
# @njit
def batchLoader(S,Y,batch_size=16):
    n_samples = S.shape[0]
    idx = np.arange(n_samples)
    np.random.shuffle(idx)
    S,Y = S[idx],Y[idx]
    for i in np.arange(0, n_samples, batch_size):
        begin, end = i, min(i + batch_size, n_samples)
        yield S[begin:end] , Y[begin:end]

In [6]:
# @njit
def updateParams(W1,b1,W2,b2,W3,b3,S,Y,batch_size=16,lr=1e-4):
    n_samples = S.shape[0]
    idx = np.arange(n_samples)
    np.random.shuffle(idx)
    S,Y = S[idx],Y[idx]
    for i in np.arange(0, n_samples, batch_size):
        begin, end = i, min(i + batch_size, n_samples)
        s,y =  S[begin:end] , Y[begin:end]
        O1,O2,O3 = forward(W1,b1,W2,b2,W3,b3,s)
        W1,b1,W2,b2,W3,b3 = backward(W1,b1,W2,b2,W3,b3,s,y,O1,O2,O3,lr)
    return W1,b1,W2,b2,W3,b3

In [7]:
# @njit
def MSELoss(y_true,y_pred):
    return np.mean(np.square(y_true-y_pred))

In [8]:
# @njit
def fitModel(X,y,n_iter=10,batch_size=16,lr=1e-4):
    W1 = np.random.uniform(low=-1/np.sqrt(X.shape[1]),high=1/np.sqrt(X.shape[1]),size=(128 ,X.shape[1]))
    b1 = np.zeros((128,1)) * 1.0
    W2 = np.random.uniform(low=-1/np.sqrt(128),high=1/np.sqrt(128),size=(256,128))
    b2 = np.zeros((256,1)) * 1.0
    W3 = np.random.uniform(low=-1/np.sqrt(256),high=1/np.sqrt(256),size=(1,256))
    b3 = np.zeros((1,1)) * 1.0
    for _ in range(n_iter):
        W1,b1,W2,b2,W3,b3 = updateParams(W1,b1,W2,b2,W3,b3,X,y,batch_size=batch_size,lr=lr)
        if _ % 50 == 0:
            print('Epoch ',_, 'Loss: ',MSELoss(y.reshape(1,-1),forward(W1,b1,W2,b2,W3,b3,X)[-1]))
    return W1,b1,W2,b2,W3,b3


In [9]:
X = np.random.rand(5000,100)
y = np.random.rand(5000,1)

In [11]:
W1,b1,W2,b2,W3,b3 = fitModel(X,y,n_iter=1000,batch_size=500,lr=1e-3)

Epoch  0 Loss:  0.08635114666072984
Epoch  50 Loss:  0.08082980509198207
Epoch  100 Loss:  0.0776415378651361
Epoch  150 Loss:  0.06797879487076072
Epoch  200 Loss:  0.06220985639052793
Epoch  250 Loss:  0.03493310705522822
Epoch  300 Loss:  0.04079295246170528
Epoch  350 Loss:  0.025483490768251054
Epoch  400 Loss:  0.02108971258172736
Epoch  450 Loss:  0.012204984298374372
Epoch  500 Loss:  0.0078330861230184
Epoch  550 Loss:  0.0066967331127153785
Epoch  600 Loss:  0.004854762148859092
Epoch  650 Loss:  0.004852825357050197
Epoch  700 Loss:  0.0032607413141964373
Epoch  750 Loss:  0.0017909979139159283
Epoch  800 Loss:  0.0026756497938496496
Epoch  850 Loss:  0.0009765741167316424
Epoch  900 Loss:  0.0012509181056471897
Epoch  950 Loss:  0.0008276321012981903
